Step 1: Determine the minimum supplier combination (greedy algorithm)
Convert different types of raw materials into "equivalent Class C materials" for easy comparison.
Use the greedy strategy to select the minimum suppliers by unit contribution so that their supply volume ≥ 24 weeks of total production demand.

Prioritize the 50 suppliers:
If any of these 50 suppliers have not supplied in the past 24 weeks, exclude these suppliers.
Calculate the equivalent supply volume of the remaining suppliers and sort them from high to low by equivalent supply volume.
If the supply volume of these 50 suppliers still cannot meet the demand:
Sort all suppliers (including the initial 50 and the remaining suppliers) by equivalent supply volume from high to low.
Use the greedy algorithm to continue selecting suppliers until the demand is met


In [3]:
import pandas as pd

# Read file
suppliers_file = r"\Users\Sample Project files\Sample Project files\telemetry-main\data\Appendix 1 Data of 402 suppliers in the past five years.xlsx"

# Read supplier order and supply data
orders_df = pd.read_excel(suppliers_file, sheet_name='The company’s order volume （m³）')
supplies_df = pd.read_excel(suppliers_file, sheet_name='Supplier’s supply volume (m³)')

# Keep only the first two columns and the last 24 columns (representing the next 24 weeks)
orders_df = orders_df.iloc[:, :2].join(orders_df.iloc[:, -24:])
supplies_df = supplies_df.iloc[:, :2].join(supplies_df.iloc[:, -24:])

# Construct Supplier ID (format: SXXX_MaterialType) and set it as the index
orders_df['Supplier ID'] = orders_df.iloc[:, 0].astype(str) + "_" + orders_df.iloc[:, 1].astype(str)
supplies_df['Supplier ID'] = supplies_df.iloc[:, 0].astype(str) + "_" + supplies_df.iloc[:, 1].astype(str)

orders_df.set_index('Supplier ID', inplace=True)
supplies_df.set_index('Supplier ID', inplace=True)

# Calculate total order quantity and total supply quantity
orders_df['Total Booking Quantity'] = orders_df.iloc[:, 2:-1].sum(axis=1)
supplies_df['Total Supply Quantity'] = supplies_df.iloc[:, 2:-1].sum(axis=1)

# Merge data to calculate supply rate
merged_df = pd.merge(
    orders_df[['Material Classification', 'Total Booking Quantity']],
    supplies_df[['Total Supply Quantity']],
    left_index=True,
    right_index=True,
    how='inner'
)
merged_df['Supply Rate'] = merged_df['Total Supply Quantity'] / merged_df['Total Booking Quantity']

# Weekly material demand and conversion parameters
weekly_demand = 28200
weeks = 24
total_demand = weekly_demand * weeks
consumption = {'A': 0.6, 'B': 0.66, 'C': 0.72}
unit_cost_ratio = {'A': 1.2, 'B': 1.1, 'C': 1.0}

# Calculate equivalent supply quantity (normalized to Material C)
merged_df['Equivalent Supply Quantity'] = merged_df.apply(
    lambda row: (row['Total Supply Quantity'] / consumption[row['Material Classification']]) * consumption['C'], axis=1
)

# ========== Modified section begins ==========
# Define a list of 50 suppliers
supplier_list = [
    'S229', 'S026', 'S106', 'S053', 'S178', 'S221', 'S237', 'S050', 'S203', 'S206',
    'S174', 'S188', 'S337', 'S175', 'S169', 'S030', 'S225', 'S122', 'S036', 'S098',
    'S076', 'S213', 'S353', 'S067', 'S016', 'S361', 'S358', 'S075', 'S014', 'S172',
    'S351', 'S193', 'S286', 'S073', 'S324', 'S282', 'S092', 'S305', 'S266', 'S152',
    'S310', 'S275', 'S392', 'S296', 'S329', 'S141', 'S027', 'S128', 'S044', 'S336'
]

# Filter data for the 50 suppliers
filtered_50_df = merged_df[merged_df.index.to_series().apply(lambda x: x.split('_')[0] in supplier_list)]

# Remove suppliers who did not supply anything in the past 24 weeks
filtered_50_df = filtered_50_df[filtered_50_df['Total Supply Quantity'] > 0]

# If the supply from the 50 suppliers is still insufficient, select from all suppliers
if filtered_50_df['Equivalent Supply Quantity'].sum() < total_demand:
    # Filter out suppliers who supplied something in the past 24 weeks from all suppliers
    all_suppliers_df = merged_df[merged_df['Total Supply Quantity'] > 0]
    # Combine the data of the 50 suppliers with all suppliers and remove duplicates
    all_suppliers_df = pd.concat([filtered_50_df, all_suppliers_df]).drop_duplicates()
    # Sort by equivalent supply quantity
    all_suppliers_df = all_suppliers_df.sort_values(by='Equivalent Supply Quantity', ascending=False)
    # Greedily select suppliers until the demand is met
    all_suppliers_df['Cumulative Equivalent Supply'] = all_suppliers_df['Equivalent Supply Quantity'].cumsum()
    needed_suppliers = all_suppliers_df[all_suppliers_df['Cumulative Equivalent Supply'] < total_demand]
    min_supplier_count = len(needed_suppliers) + 1
    selected_suppliers = all_suppliers_df.head(min_supplier_count)
else:
    # If the supply from the 50 suppliers is sufficient, directly select these 50
    filtered_50_df = filtered_50_df.sort_values(by='Equivalent Supply Quantity', ascending=False)
    filtered_50_df['Cumulative Equivalent Supply'] = filtered_50_df['Equivalent Supply Quantity'].cumsum()
    needed_suppliers = filtered_50_df[filtered_50_df['Cumulative Equivalent Supply'] < total_demand]
    min_supplier_count = len(needed_suppliers) + 1
    selected_suppliers = filtered_50_df.head(min_supplier_count)

# Output results
min_supplier_count, selected_suppliers[['Material Classification', 'Total Booking Quantity', 'Total Supply Quantity', 'Supply Rate']]

(167,
             Material Classification  Total Booking Quantity  \
 Supplier ID                                                   
 S374_C                            C                   42876   
 S229_A                            A                   31900   
 S361_C                            C                   29510   
 S108_B                            B                   34665   
 S282_A                            A                   17710   
 ...                             ...                     ...   
 S063_C                            C                     407   
 S160_C                            C                    9104   
 S135_C                            C                       2   
 S246_C                            C                    3305   
 S313_C                            C                       3   
 
              Total Supply Quantity  Supply Rate  
 Supplier ID                                      
 S374_C                       42928     1.001213  
 S229_A

               Week 241   Week 242    Week 243   Week 244   Week 245  \
Supplier ID                                                            
S001_B         0.000000   0.000000    0.000000   0.000000   0.000000   
S002_A         0.000000   0.000000    0.000000   0.000000   0.000000   
S003_C         0.000000   0.000000    1.674841   0.000000   0.000000   
S005_A        56.107182  66.156229   67.831071  75.367856  84.579483   
S007_A       110.539523  45.220714  117.238888  15.910992  25.122619   

              Week 246   Week 247   Week 248   Week 249   Week 250  ...  \
Supplier ID                                                         ...   
S001_B        0.000000   0.000000   0.000000   0.000000   0.000000  ...   
S002_A        0.000000   0.000000   0.000000   0.000000   0.000000  ...   
S003_C        0.000000   0.000000   0.000000   2.512262  14.236151  ...   
S005_A       69.505912  62.806547  67.831071  67.831071  65.318809  ...   
S007_A       19.260674  13.398730  10.049048 

Step 2: Develop the most economical ordering plan for the next 24 weeks for the selected suppliers (the last 24 columns at the end of the document are used as forecasting criteria)
Prioritize the materials with the lowest unit cost (i.e. Class C).
Evenly distribute the order quantity to each week, taking into account the past supply rate and unit material consumption of each supplier。

In [ ]:
import pandas as pd

# File path
suppliers_file = r"\Users\Sample Project files\Sample Project files\telemetry-main\data\Appendix 1 Data of 402 suppliers in the past five years.xlsx"

# Read data
orders_df = pd.read_excel(suppliers_file, sheet_name='The company’s order volume （m³）')
supplies_df = pd.read_excel(suppliers_file, sheet_name='Supplier’s supply volume (m³)')

# Keep only the first two columns and the last 24 columns (representing the next 24 weeks)
orders_df = orders_df.iloc[:, :2].join(orders_df.iloc[:, -24:])
supplies_df = supplies_df.iloc[:, :2].join(supplies_df.iloc[:, -24:])

# Construct Supplier ID
orders_df['Supplier ID'] = orders_df.iloc[:, 0].astype(str) + "_" + orders_df.iloc[:, 1].astype(str)
supplies_df['Supplier ID'] = supplies_df.iloc[:, 0].astype(str) + "_" + supplies_df.iloc[:, 1].astype(str)

orders_df.set_index('Supplier ID', inplace=True)
supplies_df.set_index('Supplier ID', inplace=True)

# Add total booking quantity and total supply quantity
orders_df['Total Booking Quantity'] = orders_df.iloc[:, 2:].sum(axis=1)
supplies_df['Total Supply Quantity'] = supplies_df.iloc[:, 2:].sum(axis=1)

# Merge to calculate supply rate
merged_df = pd.merge(
    orders_df[['Material Classification', 'Total Booking Quantity']],
    supplies_df[['Total Supply Quantity']],
    left_index=True,
    right_index=True,
    how='inner'
)

# Calculate supply rate safely
merged_df['Supply Rate'] = merged_df['Total Supply Quantity'] / merged_df['Total Booking Quantity'].replace(0, np.nan)

# Equivalent conversion
consumption = {'A': 0.6, 'B': 0.66, 'C': 0.72}
merged_df['Equivalent Supply Quantity'] = merged_df.apply(
    lambda row: (row['Total Supply Quantity'] / consumption[row['Material Classification']]) * consumption['C'], axis=1
)

# ================= Step 2: 24-week Economic Ordering Plan =================

# Select suppliers with positive equivalent supply quantity
selected_suppliers = merged_df.loc[merged_df['Equivalent Supply Quantity'] > 0].copy()

# Get column names for the 24 weeks
week_columns = supplies_df.columns[-24:]

# Extract predicted supply data for the next 24 weeks
supply_24weeks = supplies_df.loc[selected_suppliers.index, week_columns]

# Total demand and allocation ratio
weekly_demand = 28200
total_demand = weekly_demand * 24

# Calculate total supply for each supplier
supplier_total = supply_24weeks.sum(axis=1)
share = supplier_total / supplier_total.sum()
supplier_targets = share * total_demand

# Allocate weekly order quantities
weekly_ratio = supply_24weeks.div(supplier_total, axis=0)
weekly_plan = weekly_ratio.multiply(supplier_targets, axis=0)

# Rename columns to Week 241 ~ Week 264
weekly_plan.columns = [f"Week {i}" for i in range(241, 265)]

# Export or view the ordering plan
print(weekly_plan.head())
weekly_plan.to_excel("24_week_order_plan.xlsx")

               Week 241   Week 242    Week 243   Week 244   Week 245  \
Supplier ID                                                            
S001_B         0.000000   0.000000    0.000000   0.000000   0.000000   
S002_A         0.000000   0.000000    0.000000   0.000000   0.000000   
S003_C         0.000000   0.000000    1.674841   0.000000   0.000000   
S005_A        56.107182  66.156229   67.831071  75.367856  84.579483   
S007_A       110.539523  45.220714  117.238888  15.910992  25.122619   

              Week 246   Week 247   Week 248   Week 249   Week 250  ...  \
Supplier ID                                                         ...   
S001_B        0.000000   0.000000   0.000000   0.000000   0.000000  ...   
S002_A        0.000000   0.000000   0.000000   0.000000   0.000000  ...   
S003_C        0.000000   0.000000   0.000000   2.512262  14.236151  ...   
S005_A       69.505912  62.806547  67.831071  67.831071  65.318809  ...   
S007_A       19.260674  13.398730  10.049048 

Step 3: Develop the best transshipment plan for the ordering plan
Select the one with the lowest loss rate from the transshipment data.
Considering the upper limit of transshipment (6000 m³/week), try to assign a single supplier to a single transshipment.

In [ ]:
import pandas as pd

# ========== Step 1: Read forwarder data ==========
forwarder_file = r"\Users\Sample Project files\Sample Project files\telemetry-main\data\Appendix 2 Data of 8 forwarders in the past 5 years.xlsx"
loss_df = pd.read_excel(forwarder_file, sheet_name='Loss rate')

# Calculate mean loss rate for each forwarder
loss_mean = loss_df.select_dtypes(include='number').mean(axis=1)

# Extract forwarder names and sort by loss rate
forwarders = pd.DataFrame({
    'Forwarder': loss_df.iloc[:, 0],
    'LossRate': loss_mean
}).sort_values(by='LossRate').reset_index(drop=True)

print(forwarders)

# Convert to list for iteration
sorted_forwarders = forwarders['Forwarder'].tolist()

# ========== Step 2: Read the ordering plan ==========
weekly_plan = pd.read_excel("24_week_order_plan.xlsx", index_col=0)

# Extract all weeks (columns) and suppliers (index)
weeks = weekly_plan.columns.tolist()
suppliers = weekly_plan.index.tolist()

# Initialize available transport capacity for each forwarder (6000 m³ per week)
capacity = {fwd: {week: 6000 for week in weeks} for fwd in sorted_forwarders}

# Initialize transfer plan table: rows are suppliers, columns are weeks, values are selected forwarders
transfer_plan = pd.DataFrame(index=suppliers, columns=weeks)

# ========== Step 3: Assign the optimal forwarder ==========
for supplier in suppliers:
    supplier_plan = weekly_plan.loc[supplier]

    # Try to assign a "unique" optimal forwarder for this supplier
    best_fwd = None
    for fwd in sorted_forwarders:
        # Estimate if this forwarder can handle the entire 24 weeks' demand
        if all(supplier_plan[week] <= capacity[fwd][week] for week in weeks):
            best_fwd = fwd
            break

    if best_fwd:
        # Successfully assigned to a unique forwarder
        for week in weeks:
            transfer_plan.loc[supplier, week] = best_fwd
            capacity[best_fwd][week] -= supplier_plan[week]
    else:
        # Cannot assign a unique forwarder, find the optimal forwarder for each week
        for week in weeks:
            volume = supplier_plan[week]
            for fwd in sorted_forwarders:
                if capacity[fwd][week] >= volume:
                    transfer_plan.loc[supplier, week] = fwd
                    capacity[fwd][week] -= volume
                    break
            else:
                transfer_plan.loc[supplier, week] = "NoCapacity"

# ========== Step 4: Save the transfer plan ==========
transfer_plan.to_excel("24_week_forwarder_plan.xlsx")
print("Transfer allocation completed and saved to: 24_week_forwarder_plan.xlsx")

  Forwarder  LossRate
0        T3  0.090702
1        T6  0.489385
2        T4  0.667455
3        T8  0.854531
4        T2  0.921370
5        T5  0.999398
6        T1  1.904769
7        T7  2.078833
Transfer allocation completed and saved to: 24_week_forwarder_plan.xlsx
